<a href="https://colab.research.google.com/github/Mohitduklan/Deep-Learning/blob/master/SimpleRnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.2


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import re

In [5]:
# Input
article = '''The list of Odonata species of Slovenia includes 72 species of dragonflies and damselflies (Slovene: kačji pastirji) for which reliable records exist from the present-day territory of Slovenia, including one that has not been seen since the 1960s and is presumed to have been extirpated (locally extinct), but could have simply been overlooked. The list is based on two reference works: Atlas of the Dragonflies (Odonata) of Slovenia,[2] a joint publication of the Slovene Odonatological Society and the Slovene Centre for Cartography of Fauna and Flora from 1997, and the newer Atlas of the European dragonflies and damselflies (2015), supported by other, more recent publications in which new species described after 1997 were documented. Odonata species from the territory of present-day Slovenia were systematically studied by the naturalists Johann Weikhard von Valvasor and Giovanni Antonio Scopoli as early as the 17th and 18th centuries; however, the first systematic compendium was only published in the 1960s by the Slovene zoologist Boštjan Kiauta [sl].[1] The distribution of Odonata in Slovenia is now fairly well known by international standards, with Slovenia having been one of the first European countries for which a full account of faunistic data (an "atlas") was published. The number of species (72) represents almost exactly half of the European species (143) and is comparable with the number of species of Germany (81) and Spain (80), both much larger countries.[3] Slovenian odonate fauna is therefore considered highly diverse and is attributed to the country's position on the junction of several ecoregions where many species reach the border of their distribution.'''

In [6]:
# Preprocessing
article = article.lower() # Lower
article = re.sub("[^a-z .']+", " ", article) # Removing special characters
article = re.sub("[^a-z .]+", "", article).split() # Spliting word

In [10]:
# Initializing word to id and id to word dictionary
vocab = list(set(article))
vocab_size = list(set(article))
word2id = {v: w for w, v in enumerate(vocab)}
id2word = {w: v for w, v in enumerate(vocab)}

In [12]:
# Vectorizing text and saving in tf dataset
SEQUENCE_LENGTH = 5
BATCH_SIZE = 8
BUFFER_SIZE = 10000
examples_per_epoch = len(article)//(SEQUENCE_LENGTH+1)

article_vector = [word2id[x] for x in article]
dataset = tf.data.Dataset.from_tensor_slices(article_vector)
dataset = dataset.batch(SEQUENCE_LENGTH+1, drop_remainder=True)
dataset = dataset.map(lambda x: [x[:-1], x[1:]])
dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

<PrefetchDataset shapes: ((8, 5), (8, 5)), types: (tf.int32, tf.int32)>

In [13]:
# Building model
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(
            rnn_units, 
            return_sequences=True,
            return_state=True
        )
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [16]:
# Model parameters
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = 'adam'
model = MyModel(vocab_size, embedding_dim, rnn_units)
model.compile(optimizer=optimizer, loss=loss)

In [ ]:
history = model.fit_generator(dataset, epochs=20)

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


5/5 [==============================] - 2s 162ms/step - loss: 4.9726
Epoch 2/20
5/5 [==============================] - 1s 160ms/step - loss: 4.8784
Epoch 3/20
5/5 [==============================] - 1s 158ms/step - loss: 4.6701
Epoch 4/20
5/5 [==============================] - 1s 159ms/step - loss: 4.5051
Epoch 5/20
5/5 [==============================] - 1s 170ms/step - loss: 4.1682
Epoch 6/20
5/5 [==============================] - 1s 160ms/step - loss: 4.0942
Epoch 7/20
5/5 [==============================] - 1s 157ms/step - loss: 3.8393
Epoch 8/20
5/5 [==============================] - 1s 154ms/step - loss: 3.5601
Epoch 9/20
5/5 [==============================] - 1s 154ms/step - loss: 3.2540
Epoch 10/20
5/5 [==============================] - 1s 161ms/step - loss: 2.8043
Epoch 11/20
5/5 [==============================] - 1s 155ms/step - loss: 2.4274
Epoch 12/20
5/5 [==============================] - 1s 157ms/step - loss: 2.1217
Epoch 13/20
5/5 [==============================] - 1s 159ms/

In [ ]:
p = model.predict(article_vector[8:16])

In [ ]:
[id2word[np.argmax(i[0])] for i in p]

['of', 'the', 'and', 'damselflies', 'slovene', 'zoologist', 'ji', 'list']